In [3]:
import torch
import torchvision as tv
import torchvision.transforms as transforms
import torch.nn as nn
import argparse
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd


BATCH_SIZE = 64
LR = 0.001
EPOCH = 5

#  获取数据

In [4]:
transform = transforms.ToTensor()
trainset = tv.datasets.MNIST('./data/',train=True,transform=transform,download=True)
testset = tv.datasets.MNIST('./data/',train=False,transform=transform,download=True)

# 数据预处理

In [5]:
trainloader = torch.utils.data.DataLoader(trainset,batch_size=BATCH_SIZE,shuffle=True)
testloader = torch.utils.data.DataLoader(trainset,batch_size=BATCH_SIZE,shuffle=False)

# 模型搭建

In [6]:
class LeNet(nn.Module):
    def __init__(self):
        super(LeNet,self).__init__()
        self.conv1 = nn.Sequential(    
            nn.Conv2d(in_channels=1,out_channels=6,kernel_size=5,stride=1,padding=2),#卷积
            nn.BatchNorm2d(6),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,stride=2) #池化
            )
        self.conv2 = nn.Sequential(
            nn.Conv2d(in_channels=6,out_channels=16,kernel_size=5,stride=1),#卷积
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,stride=2) #池化
            )
        self.fc1 = nn.Sequential(
            nn.Linear(5*5*16,120),
            nn.ReLU())
        self.fc2 = nn.Sequential(
            nn.Linear(120,84),
            nn.ReLU())
        self.fc3 = nn.Linear(84,10)
        
    def forward(self,x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = x.view(x.size()[0],-1)
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc3(x)
        return x

# 损失函数及优化方式

In [7]:
net =LeNet()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(),lr = LR,betas=(0.9,0.999),eps=1e-08,weight_decay=0)


# 训练模型及评估

In [8]:
def train():
    for epoch in range(EPOCH):
        sum_loss = 0.0
        for i,data in enumerate(trainloader):
            inputs,labels = data
            optimizer.zero_grad()
            outputs = net(inputs)
#             print(outputs.shape)
            loss = criterion(outputs,labels)
            loss.backward()
            optimizer.step()
            
            #没训练200个batch打印一次平均loss
            sum_loss +=loss.item()
            if i%200 == 199:  
                print('[%d,%d] loss:%.03f' % (epoch+1,i+1,sum_loss /200))
                sum_loss =0.0
                
    with torch.no_grad():
        correct=0
        total=0
        for data in testloader:
            images,labels = data
            outputs = net(images)
#             print("out is :\n",outputs[0][0])
#             print("outputs.data is :",outputs.data[0][0])
            _,predicted = torch.max(outputs.data,1)
            total += labels.size(0)
            correct +=(predicted ==labels).sum()
        print('第%d个epoch的识别准确率为:%.2f%%' %(epoch+1,(100.0* correct/total)))

In [9]:
if __name__ == "__main__":
    train()

[1,200] loss:0.440
[1,400] loss:0.123
[1,600] loss:0.079
[1,800] loss:0.067
[2,200] loss:0.060
[2,400] loss:0.058
[2,600] loss:0.054
[2,800] loss:0.052
[3,200] loss:0.044
[3,400] loss:0.041
[3,600] loss:0.042
[3,800] loss:0.042
[4,200] loss:0.033
[4,400] loss:0.035
[4,600] loss:0.026
[4,800] loss:0.034
[5,200] loss:0.026
[5,400] loss:0.031
[5,600] loss:0.023
[5,800] loss:0.028
第5个epoch的识别准确率为:99.27%
